In [1]:
import os
import sys

# set environment path
sys.path.append("../..")

# data manipulation
import numpy as np
import pandas as pd
import warnings

from gensim.models.doc2vec import Doc2Vec
from sqlalchemy import create_engine
from tqdm import tqdm

# custom functions
from src.models import algorithms, evaluate_model
from src.utilities import utilities

tqdm.pandas()
warnings.filterwarnings('ignore')

/Users/jensen/Desktop/recommendation-systems/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data and Models

In [2]:
# set path to where repo is located
REPO_PATH = os.path.join("..", "..")

# setting path
DATA_PATH = os.path.join(REPO_PATH, "data", "evaluation")
D2V_PATH = os.path.join(REPO_PATH, "models", "d2v")

# category to train
CATEGORY = "Grocery_and_Gourmet_Food"

# reproducibility checks
SEED = 42
np.random.seed(42)

# load train/test dataset and doc2vec trained model
train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")
d2v = Doc2Vec.load(f"{D2V_PATH}/{CATEGORY}_item_50_10_d2v.model")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A23RYWDS884TUL,5.0,This curry paste makes a delicious curry. I j...,2013-05-28,curry paste delicious curry fry chicken vegeta...
1,1,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A945RBQWGZXCK,5.0,I've purchased different curries in the grocer...,2012-09-17,purchase different curry grocery store complet...
2,3,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3AMNY44OP8AOU,4.0,I started a new diet restricting all added sug...,2014-01-23,start new diet restrict added sugar brand suga...
3,4,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3IB4CQ2QEJLJ8,5.0,So many flavors. I can't begin to tell you how...,2014-04-27,flavor begin tell love mae ploy curry ask reci...
4,5,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",AQA5DF3RWKETQ,5.0,I've used this a lot recently in some of my ch...,2012-11-27,use lot recently chicken dish use lot like spi...
47769,77420,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A192LQZWDYPR4U,5.0,Another quality Reese Peanut Butter Cup produc...,2014-02-27,quality reese peanut butter cup product great ...
47770,77421,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A2QKXW3LDQ66P5,5.0,I purchased these for my husband who has every...,2013-02-20,purchase husband love reeses valentine day pre...
47771,77430,B00ID9VSOM,"Viva Labs Organic Coconut Sugar: Non-GMO, Low-...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2P3TGJU301KXD,5.0,this stuff is INCREDIBILY yummy! SO much bette...,2014-07-15,stuff incredibily yummy good regular brown sug...
47772,77456,B00IRL93SY,Barrie House Kenya Estate - AA Single Cup Caps...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AEFE9VDHTQ199,5.0,"Very nice aroma, body and taste! Will buy this...",2014-05-24,nice aroma body taste buy coffee good coffee a...
47773,77508,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2AEZQ3DGBBLPR,2.0,This is a no go for diabetics according to my ...,2014-06-26,diabetic accord wife doctor order intention us...


In [4]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,2,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1TCSC0YWT82Q0,5.0,I love ethnic foods and to cook them. I recent...,2013-08-03,love ethnic food cook recently purchase produc...
1,8,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1Z7Y2GMAP9SRY,5.0,I like to make my own curry but this is a tast...,2014-06-27,like curry tasty alternative use base kind dif...
2,23,B00004S1C5,"Ateco Food Coloring Kit, 6 colors","['Grocery & Gourmet Food', 'Cooking & Baking',...",A14YSMLYLJEMET,1.0,This product is no where near natural / organi...,2013-03-29,product near natural organic wish review purch...
3,31,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",A2F488C4PLWGEI,5.0,If my wife drinks a cup of this tea when she f...,2014-03-23,wife drink cup tea feel attack come help avoid...
4,32,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AO1HXV7DWZZIR,5.0,I don't know about the medicinal aspects of th...,2014-02-06,know medicinal aspect tea flavor downright scr...
28001,77519,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A1WT3TVHANP7ZF,3.0,Hmmm. I really wanted to love this sweetener. ...,2014-07-22,hmmm want love sweetener half sugar half stevi...
28002,77520,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A3NEAETOSXDBOM,5.0,"I confess I have a sweet tooth, and love the t...",2014-06-30,confess sweet tooth love taste sugar recognize...
28003,77521,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",AD1ZOPB0BBEHB,4.0,"It has a little of the stevia aftertaste, but ...",2014-07-17,little stevia aftertaste fair compromise able ...
28004,77522,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A18ECVX2RJ7HUE,5.0,i love marinade for grilled flank steak or lon...,2014-05-30,love marinade grilled flank steak london broil...
28005,77523,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2G04D4QZAXL15,3.0,I've been using Truvia (a form of stevia) on m...,2014-05-27,use truvia form stevia cereal greek yogurt yea...


In [5]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))
print(test_user_history)

                  reviewerID  \
0      A00177463W0XWB16A9O05   
1      A022899328A0QROR32DCT   
2      A068255029AHTHDXZURNU   
3      A06944662TFWOKKV4GJKX   
4             A1004703RC79J9   
...                      ...   
13274          AZWRZZAMX90VT   
13275          AZXKAH2DE6C8A   
13276          AZXON596A1VXC   
13277          AZYXC63SS008M   
13278          AZZ5ASC403N74   

                                                    asin  
0                               [B00474OR8G, B00BFM6OAW]  
1                                           [B00CMQDKES]  
2                                           [B001FA1K2G]  
3                                           [B000GFYRHG]  
4                                           [B003GTR8IO]  
...                                                  ...  
13274  [B0007R9L4M, B000CN7BMA, B001EQ5D1K, B002VT3GX...  
13275   [B000MAK41I, B004X8TJP2, B006H34CUS, B007W14RMM]  
13276                           [B001EO5S0I, B00271QQ7Q]  
13277                    

In [6]:
# testing d2v models
d2v.dv[0]

array([-0.19320773,  0.23249052,  0.10087004,  0.11353737, -0.29637456,
        0.1206838 , -0.15905488, -0.03072077, -0.18343581,  0.09816217,
       -0.01866587, -0.02012681, -0.49991536,  0.05031789,  0.2729918 ,
       -0.14091071,  0.43504936, -0.36140546, -0.08976636, -0.06389496,
       -0.09563986, -0.0033619 ,  0.38012826, -0.10016942, -0.07466158,
        0.6029501 , -0.3629382 , -0.02172032, -0.37396008,  0.03635319,
        0.22457832,  0.36284095, -0.09430069,  0.5297421 , -0.04680703,
        0.17874746, -0.27136478, -0.16296121,  0.21850628,  0.12752077,
       -0.09792925, -0.17686078, -0.22154953, -0.29303235,  0.23883624,
       -0.09763848,  0.02103806, -0.09468511,  0.02304819,  0.1887425 ],
      dtype=float32)

# Generate N-Recommendations = {5, 10, 15, 20}

## Instantiate Embedded Review CBF (Item-based)

In [7]:
er_cbf = algorithms.EmbeddedReviewCBF(d2v)

In [8]:
%%time
# fit learning algorithm to training data
er_cbf.fit(train)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 13397/13397 [00:00<00:00, 39813.72it/s]

CPU times: user 507 ms, sys: 25.3 ms, total: 532 ms
Wall time: 532 ms


In [9]:
%%time
# generate n-number of candidates items (200)
candidate_items = er_cbf.test()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 13397/13397 [00:05<00:00, 2587.19it/s]

CPU times: user 16.9 s, sys: 3.09 s, total: 20 s
Wall time: 5.18 s


## Loop through N = {5, 10, 15, 20}

For each top-N setting, we will generate candidates items up to *N*-items and run metrics evaluation of `Recall@N` and `Novelty@N` on all users.

In [10]:
# generate item popularity
item_popularity = evaluate_model.generate_item_popularity(train)

In [11]:
%%time
n_recommendations = {}
for n in [5, 10, 15, 20]:
    # retrieve the top-n items based on similarities
    top_ns = er_cbf.get_top_n(n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name="ER-CBF",
                                                    top_ns=top_ns,
                                                    user_rating_history=test_user_history, 
                                                    item_popularity=item_popularity, 
                                                    n=n,
                                                    mf_based=False))
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

The ER-CBF has an average recall@5: 0.01361, average novelty@5: 0.93771
The ER-CBF has an average recall@10: 0.02177, average novelty@10: 0.94207
The ER-CBF has an average recall@15: 0.02927, average novelty@15: 0.94574
The ER-CBF has an average recall@20: 0.03584, average novelty@20: 0.94881
CPU times: user 48.3 s, sys: 657 ms, total: 49 s
Wall time: 48.8 s


# Evaluate N-Recommendations

For each top-N recommendation list, we pick a randomly sampled user to look at their *N*-number of recommendations based on their purchase history.

## N=5

In [12]:
top_ns_05 = n_recommendations[5][0]

In [13]:
utilities.retrieve_recommendations(train, top_ns_05)

For user: A2H84V55USFJQX:
Purchase History:
             asin                                              title
13844  B000ZSZ5S4  Blue Diamond Almonds, Bold Salt &amp; Vinegar,...
41088  B005K4Q1T0  Grove Square Hot Cocoa Dark Chocolate, 24 Sing...

Recommending:

         asin                                              title
0  B0033HPPIO  Donut House Collection Coffee Chocolate Glazed...
1  B001EO5Z6A   Rocamojo Organic Gourmet Coffee, Wild French ...
2  B002WENMX4  Suchard Hot Chocolate Syrup, 8-Count T-Discs f...
3  B002MV63EW  Gloria Jean's K-Cup, Holiday Traditions, 24-Co...
4  B005QKH1V0  Cameron's Coffee Roasted Ground Coffee, Variet...


## N=10

In [14]:
top_ns_10 = n_recommendations[10][0]

In [15]:
utilities.retrieve_recommendations(train, top_ns_10)

For user: A2W9B725TZBXOX:
Purchase History:
             asin                                              title
1645   B0005Z6LLW  Ghirardelli Hot Chocolate Mix , Chocolate Moch...
4683   B000ED9L6C  Bob's Red Mill Raw Shelled Sunflower Seeds (Ke...
8508   B000GATCRQ  Dream Foods International Volcano Lemon Burst,...
15333  B0014EOUAW  V8 V-Fusion Light Peach Mango Juice Drink, 46-...
18192  B001E52ZAS  Post Shredded Wheat Lightly Frosted Cereal, Sp...
21093  B001EPQV1W  Honey Bunches of Oats with Almonds, 14.5-Ounce...
22932  B001G8UC8K  Tootsie Roll Midgees Candy 5 Pound Value Bag 7...
23025  B001GVIRD4                     Grocery &amp; Gourmet Food" />
24945  B001NC8HQS  Idahoan Buttery homestyle flavored mashed pota...
25830  B001SAQ7WQ  Progresso Traditional Soup, Chickarina (Chicke...
26956  B0025UOMY8  Maruchan Yakisoba Spicy Vegetable, 3.79-Ounce ...
33467  B00421DMLM  Libby's Splenda Sliced Peaches, 23.5-Ounce Jar...
34141  B004727CL2  Snack Factory Pretzel Crisps Everything,

## N=15

In [16]:
top_ns_15 = n_recommendations[15][0]

In [17]:
utilities.retrieve_recommendations(train, top_ns_15)

For user: A1G8ECTOF35O2R:
Purchase History:
             asin                                              title
9115   B000H26J7E  Lindt Excellence Bar, 70% Cocoa Smooth Dark Ch...
47677  B00EKLPLU4              Healthworks Cacao Powder Organic, 1lb

Recommending:

          asin                                              title
0   B0019ZHU2Q  Ghirardelli Chocolate Intense Dark Bar, Twilig...
1   B000KNB4WA  Ghirardelli Chocolate Baking Chips, Semi-Sweet...
2   B001G0MG2I   Ghirardelli Chocolate Intense Dark Bar, Midni...
3   B000OQ4A3S  Navitas Organics Cacao Nibs, 8 oz. Bag &mdash;...
4   B000Y0HCHK  Nevada Manna Sugar Free Semi Sweet Chocolate C...
5   B000H27PB8  Ritter Sport Bars, Dark Chocolate with Marzipa...
6   B0029J6QLM         Snickers Dark Chocolate Candy (Pack of 24)
7   B0029JASWA  Dove Dark Chocolate Promises, 9.5-Ounce Packag...
8   B000JSLYSU        Cadbury Flake Chocolate Bars 23.5g, 4-Count
9   B001H9N9RE  Ghirardelli Dark Chocolate Twilight Delight 72...
10  B00

## N=20

In [18]:
top_ns_20 = n_recommendations[20][0]

In [19]:
utilities.retrieve_recommendations(train, top_ns_20)

For user: A1NEKFNO1E743Y:
Purchase History:
            asin                                              title
8695  B000GFYRHG  Bigelow Constant Comment Tea 20-Count Boxes (P...

Recommending:

          asin                                              title
0   B000F4DKAI  Twinings of London English Afternoon Black Tea...
1   B007JFXXJY  Lipton Herbal Pyramid Tea Bags, Orange Blossom...
2   B001E5E1WS  Rishi Tea Organic Silver Needle Jasmine Loose ...
3   B005CT9OEW  Lipton Green Tea Bags, Decaffeinated Blackberr...
4   B001BM62T4  Red Espresso Ground Rooibos Tea, 8.8-Ounce Pou...
5   B008YUL4CQ  Lipton Decaffeinated Green Tea, Superfruit Jas...
6   B002AR158W   Numi Organic Tea Magnolia Puerh, Full Leaf Gr...
7   B0061IUKDM  Higgins &amp; Burke, Green Tea, 20 Count (Pack...
8   B001E5E20O  Rishi Tea Organic Black Tea, Earl Grey Loose T...
9   B004051BO0  Yogi Tea - Vanilla Spice Perfect Energy - Ener...
10  B002HQCWYM  Twinings of London English Breakfast Tea K-Cup...
11  B000CQBZ

# Cross-Analysis for Cold-Start Users (<= 2 Purchased Items)

For each top-N setting, we will generate candidates items up to *N*-items and run metrics evaluation of `Recall@N` and `Novelty@N` on cold-start users (e.g., users who purchased two or less items based on items per user in the training set).

In [20]:
cold_start_users = utilities.generate_cold_start_users(train)

In [21]:
for n in tuple(zip([5, 10, 15, 20], [top_ns_05, top_ns_10, top_ns_15, top_ns_20])):
    cold_start_top_ns = dict(filter(lambda x: x[0] in cold_start_users, n[1].items()))
    # evaluate how well the recommended items predicted the future purchases
    # on cold start users
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name="ER-CBF",
                                                    top_ns=cold_start_top_ns,
                                                    user_rating_history=test_user_history, 
                                                    item_popularity=item_popularity, 
                                                    n=n[0],
                                                    mf_based=False))

The ER-CBF has an average recall@5: 0.01585, average novelty@5: 0.93636
The ER-CBF has an average recall@10: 0.02408, average novelty@10: 0.94052
The ER-CBF has an average recall@15: 0.03251, average novelty@15: 0.94431
The ER-CBF has an average recall@20: 0.03967, average novelty@20: 0.94768


# Generating Recommended Items DataFrame

In [22]:
max_recommendations = (utilities
                       .generate_recommendations_df(train=train,
                                                    n_recommendations=n_recommendations,
                                                    algo_name="ER-CBF",
                                                    mf_based=False,
                                                    max_recommended=20))

In [23]:
max_recommendations

,reviewerID,item_rank,asin,algorithm,title
0,A00177463W0XWB16A9O05,0,B001D0IZBM,ER-CBF,"Grocery &amp; Gourmet Food"" />"
1,A00177463W0XWB16A9O05,1,B000J4IDO2,ER-CBF,"Coffee People K-Cups, Black Tiger Extra Bold, ..."
2,A00177463W0XWB16A9O05,2,B001ELL67A,ER-CBF,"Grocery &amp; Gourmet Food"" />"
3,A00177463W0XWB16A9O05,3,B005K4Q1YA,ER-CBF,"Grove Square Cappuccino, French Vanilla, 24 Co..."
4,A00177463W0XWB16A9O05,4,B0007IQQXA,ER-CBF,Van Houtte French Vanilla Coffee K Cups 18 Count
...,...,...,...,...,...
267935,AZZ5ASC403N74,15,B00DBSG2HS,ER-CBF,"Keebler&nbsp;Cookies, Simply Made,&nbsp;Chocol..."
267936,AZZ5ASC403N74,16,B000IMWNCI,ER-CBF,"HERSHEY'S Kitchens Baking, Mildly Sweet Dark C..."
267937,AZZ5ASC403N74,17,B004DOXI9O,ER-CBF,"Lazy Day Foods Dark Chocolate Ginger Cookies, ..."
267938,AZZ5ASC403N74,18,B001OCBT3U,ER-CBF,"Planters Big Nut Bars, Double Peanut, 8.1-Ounc..."


# Store in `SQLite` DB

In [24]:
# engine = create_engine("sqlite:///recommender.db", echo=True)

In [25]:
# max_recommendations.to_sql(f"{CATEGORY}", con=engine, if_exists="append")